# Data Exploration for Yelp Dataset

This is a notebook that explores the sample Yelp dataset provided by Kaggle (https://www.kaggle.com/yelp-dataset/yelp-dataset).
 
The datasets are as follows:
- **yelp_academic_dataset_business.json**: TBD
- **yelp_academic_dataset_checkin.json**: TBD
- **yelp_academic_dataset_review.json**: TBD
- **yelp_academic_dataset_tip.json**: TBD
- **yelp_academic_dataset_user.json**: TBD

### Imports and Parameters

In [ ]:
pip install findspark py4j ipywidgets matplotlib seaborn

In [ ]:
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
import findspark
findspark.init("/Users/ana/server/spark-3.0.1-bin-hadoop2.7")

import pyspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import isnan, when, count, col
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from matplotlib import pyplot as plt
%matplotlib inline
#Seaborn for data visualization
import seaborn as sns

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [ ]:
sc = pyspark.SparkContext(appName="data-exploration-yelp")

spark = SparkSession(sc)

In [ ]:
yelp_business = "./yelp-raw/yelp_academic_dataset_business.json"

yelp_checkin = "./yelp-raw/yelp_academic_dataset_checkin.json"

yelp_review= "./yelp-raw/yelp_academic_dataset_review.json"
    
yelp_tip= "./yelp-raw/yelp_academic_dataset_tip.json"

yelp_user= "./yelp-raw/yelp_academic_dataset_user.json"

### Initial Data Exploration

We are going to explore each dataset and understand various statistical analysis of the data.

#### Business Dataset Overview

In [ ]:
business_df = spark.read.json(yelp_business)

In [ ]:
business_df.show(5,False)

In [ ]:
business_df.printSchema()

In [ ]:
business_df.count()

#### Flatten the dataset

In [ ]:
# Flattends dataframe to remove nesting.
flat_business_df = business_df.select("address", "attributes.*", "business_id", "categories", "city", "hours.*", 
                                      "is_open", "latitude", "longitude", "name", "postal_code", "review_count", 
                                      "stars", "state")

In [ ]:
flat_business_df.show(5, False)

In [ ]:
flat_business_df.printSchema()

In [ ]:
flat_business_df.write.parquet("./yelp_transformed/business_v1")

In [ ]:
transformed_business_df = spark.read.parquet("./yelp_transformed/business_v1")

In [ ]:
transformed_business_df.printSchema()

In [ ]:
transformed_business_df.show(20, False)

In [ ]:
# Get count of all the rows to get the amount of business.
total_businesses = transformed_business_df.count()
total_businesses

In [ ]:
# Creates new dataframe that counts all nulls within a column iteratively.
null_df = transformed_business_df.select([count(when(col(c).isNull(), c)).alias(c) for c in transformed_business_df.columns])

In [ ]:
null_pd = null_df.toPandas()

In [ ]:
null_pd

In [ ]:
# Divide all null column counts by the amount of businesses, to get total percentage of missing values
null_percentage_pd = null_pd.div(total_businesses / 100, axis=0)
null_percentage_pd

In [ ]:
# Filter null percentages dataframe by set percentage. Here we default by 50%, as we want to try our first pass 
# of training with relatively stable columns.

percentage_to_filter_by = 50

null_filtered_pd = null_percentage_pd[null_percentage_pd < percentage_to_filter_by]
null_filtered_na = null_filtered_pd.dropna(axis=1, how='all')
null_filtered_na

In [ ]:
#Filter out weekday opening time columns, as they are not going to be very useful for model recommendations.

final_null_pd = null_filtered_na.drop(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"], axis=1)
final_null_pd

In [ ]:
columns_to_keep = final_null_pd.columns.tolist()
columns_to_keep

In [ ]:
filtered_business_df = transformed_business_df.select(*columns_to_keep)
filtered_business_df.show(5, False)

In [ ]:
dropdown_columns = widgets.Dropdown(options = filtered_business_df.columns)

output_column = widgets.Output()

def dropdown_columns_event_handler(change):
    output_column.clear_output()
    
    filtered_business_df.groupBy(change.new).count().sort(col("count").desc()).show(50, False)

dropdown_columns.observe(dropdown_columns_event_handler, names='value')
display(dropdown_columns)

### Data Visualization Exploration

In [ ]:
stars_count_df = filtered_business_df.groupBy(col("stars")).count().sort(col("count").desc())
stars_count_df.show(10, False)

In [ ]:
stars_pd = stars_count_df.toPandas()
stars_pd.head(10)

In [ ]:
sns.catplot(x="stars", y="count", kind="bar", data=stars_pd)

plt.show()